In [1]:
import librosa as lb
import os
import numpy as np
import matplotlib.pyplot as plt

C:\anaconda\envs\epsilon\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\anaconda\envs\epsilon\lib\site-packages\numpy\.libs\libopenblas.4SP5SUA7CBGXUEOC35YP2ASOICYYEQZZ.gfortran-win_amd64.dll
C:\anaconda\envs\epsilon\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [1]:
train_audio_path="train/audio/"

In [2]:
labels=os.listdir(train_audio_path)
labels

NameError: name 'os' is not defined

In [ ]:
all_wave=[]
all_label=[]
for label in labels:
    print(label)
    waves=[f for f in os.listdir(train_audio_path+'/'+label) if f.endswith('.wav')]
    for wave in waves:
        sampels,sample_rate=lb.load(train_audio_path+'/'+label+'/'+wave,sr=16000)
        sampels=lb.resample(sampels,orig_sr=sample_rate,target_sr=8000)
        if len(sampels)==8000:
            all_wave.append(sampels)
            all_label.append(label)

bed
bird
cat
dog
down
eight
five
four
go
happy
house
left
marvin
nine
no
off
on
one
right
seven
sheila
six
stop
three
tree
two
up


In [ ]:
x=[]
for i in all_wave:
    x.append(np.mean(lb.feature.mfcc(y=i,sr=len(all_wave[0]),n_mfcc=150).T,axis=0))


In [ ]:
#import pandas as pd
#df=pd.DataFrame(x)


In [ ]:
#df["Target"]=all_label

In [ ]:
#df

In [ ]:
#df.Target.value_counts()

In [ ]:
#import seaborn as sns

In [ ]:
#plt.figure(figsize=(30,15))
#sns.countplot(x=df["Target"])

In [ ]:
#x=df.drop(columns="Target",axis=1)
#y=df["Target"]
#from sklearn.model_selection import train_test_split

In [ ]:
#x_train,x_test,y_train,y_test=train_test_split(x,y,stratify=y,shuffle=True,test_size=0.2)

In [ ]:
'''print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)'''

In [41]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
y=le.fit_transform(all_label)
set(y)
classes=list(le.classes_)
from keras.utils import np_utils
# Convert labels to categorical variables
y = np_utils.to_categorical(y)

In [49]:
all_wave=np.array(all_wave,dtype='uint8').reshape(-1,8000,1)

In [50]:
all_wave.shape

(58252, 8000, 1)

In [53]:
y.shape

(58252, 30)

In [134]:
#all_label=np.array(all_label)

In [54]:
from sklearn.model_selection import train_test_split

In [55]:
x_train,x_val,y_train,y_val=train_test_split(all_wave,y,stratify=y,shuffle=True,test_size=0.2)

In [56]:
print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)

(46601, 8000, 1)
(46601, 30)
(11651, 8000, 1)
(11651, 30)


In [58]:
import tensorflow as tf

In [59]:
from tensorflow import keras


In [ ]:

model=keras.Sequential([
    #first
    keras.layers.Conv1D(8,13,activation="relu",input_shape=(8000,1)),
    keras.layers.MaxPooling1D(3),
    #second
    keras.layers.Conv1D(16,11,activation="relu"),
    keras.layers.MaxPooling1D(3),
    #third
    keras.layers.Conv1D(16,11,activation="relu"),
    keras.layers.MaxPooling1D(3),
    #forth
    keras.layers.Conv1D(64,7,activation="relu"),
    keras.layers.MaxPooling1D(3),
    #fifth
    keras.layers.Conv1D(16,11,activation="relu"),
    keras.layers.MaxPooling1D(3),
    #flatten()
    keras.layers.Flatten(),
    #Dense layer1
    keras.layers.Dense(256,activation="relu"),
    #Dense2
    keras.layers.Dense(128,activation="relu"),
    #output
    keras.layers.Dense(30,activation="softmax")
])

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=["accuracy"])

In [ ]:
model.fit(x_train[0:2000],y_train[0:2000],epochs=5)

## mmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmm

In [50]:
def predict(audio):
    probabilities=model.predict(audio.reshape(1,8000,1))
    index=probabilities.argmax()
    return classes[index]   

In [34]:
predict(x_val[0])

1/1 [==============================] - 0s 288ms/step


'go'

In [35]:
y_val[0]

array([0., 0., 0., 0., 0., 0., 1., 0., 0.], dtype=float32)

In [44]:
sampels,sample_rate=lb.load(r'D:\speech recognition\tensorflow-speech-recognition-challenge\test\audio\clip_0000adecb.wav',sr=8000)

In [52]:
import IPython.display as ipd

In [86]:
ipd.Audio(sampels,rate=8000)

In [155]:
print(predict(audio_data))

1/1 [==============================] - 0s 75ms/step
go


In [93]:
import soundfile as sf
import sounddevice as sd

In [96]:
mydata=sd.rec(16000,channels=1)

In [97]:
mydata

array([[ 0.0000000e+00],
       [ 0.0000000e+00],
       [-3.0517578e-05],
       ...,
       [ 0.0000000e+00],
       [ 0.0000000e+00],
       [ 0.0000000e+00]], dtype=float32)

In [149]:
# Set the recording parameters
duration =2# seconds
sample_rate =4000
channels = 1
filename = "output.wav"

# Start recording
print("Recording...")
recording = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=channels)

# Wait for recording to finish
sd.wait()

# Save the recording to a WAV file
sf.write(filename, recording, sample_rate)
print(f"Recording saved to {filename}")

Recording...
Recording saved to output.wav


In [150]:
sampels,sample_rate=lb.load("output.wav")

In [152]:
ipd.Audio(sampels,rate=8000)

In [153]:
# Load the saved audio file
filename = "output.wav"
audio_data, sample_rate = sf.read(filename)

# Get the shape of the audio data
shape = np.shape(audio_data)

# Print the shape
print("Shape of audio data:", shape)

Shape of audio data: (8000,)
